# Setup

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
######## GPU CONFIGS FOR RTX 2070 ###############
## Please ignore if not training on GPU       ##
## this is important for running CuDNN on GPU ##

tf.keras.backend.clear_session() #- for easy reset of notebook state

# chck if GPU can be seen by TF
tf.config.list_physical_devices('GPU')
#tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
###############################################

1 Physical GPUs, 1 Logical GPU


# Generating Text with GPT

In [5]:
from transformers import TFOpenAIGPTLMHeadModel, OpenAIGPTTokenizer

In [8]:
gpttokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')
gpt = TFOpenAIGPTLMHeadModel.from_pretrained('openai-gpt')

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [9]:
input_ids = gpttokenizer.encode('Robotics is the domain of ', return_tensors='tf')
print(input_ids)
greedy_output = gpt.generate(input_ids, max_length=100)

print("Output:\n" + 100 * '-')
print(gpttokenizer.decode(greedy_output[0], skip_special_tokens=True))

tf.Tensor([[ 5846  9259   544   481 15611   498]], shape=(1, 6), dtype=int32)
Output:
----------------------------------------------------------------------------------------------------
robotics is the domain of the government. " 
 " i see. " 
 " i'm sure you do. " 
 " i'm sure you do. " 
 " i'm sure you do. " 
 " i'm sure you do. " 
 " i'm sure you do. " 
 " i'm sure you do. " 
 " i'm sure you do. " 
 " i'm sure you do. " 
 " i'm sure you do. " 
 " i


# Generating Text with GPT-2

In [3]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

gpt2tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
gpt2 = TFGPT2LMHeadModel.from_pretrained("gpt2", 
                                         pad_token_id=gpt2tokenizer.eos_token_id)

In [18]:
# encode context the generation is conditioned on
input_ids = gpt2tokenizer.encode('Robotics is the domain of ', return_tensors='tf')

# generate text until the output length (which includes the context length) reaches 50
greedy_output = gpt2.generate(input_ids, max_length=50)

print("Output:\n" + 50 * '-')
print(gpt2tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
--------------------------------------------------
Robotics is the domain of the United States Government.

The United States Government is the primary source of information on the use of drones in the United States.

The United States Government is the primary source of information on the use of drones


In [26]:
# BEAM SEARCH
# activate beam search and early_stopping
beam_output = gpt2.generate(
    input_ids, 
    max_length=50, 
    num_beams=20, 
    early_stopping=True
)

print("Output:\n" + 50 * '-')
print(gpt2tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
--------------------------------------------------
Robotics is the domain of science, technology, engineering, mathematics, and mathematics (STEM). It is the domain of science, technology, engineering, mathematics, and mathematics (STEM). It is the domain of science, technology, engineering, mathematics,


In [24]:
# set no_repeat_ngram_size to 3
beam_output = gpt2.generate(
    input_ids, 
    max_length=50, 
    num_beams=5, 
    no_repeat_ngram_size=3, 
    early_stopping=True
)

print("Output:\n" + 50 * '-')
print(gpt2tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
--------------------------------------------------
Robotics is the domain of science and technology.

In this article, we will look at some of the most important aspects of robotics and how they can be used to improve the lives of people around the world. We will also look at the


In [36]:
# Returning multiple beams
beam_outputs = gpt2.generate(
    input_ids, 
    max_length=50, 
    num_beams=7, 
    no_repeat_ngram_size=3, 
    num_return_sequences=3,  
    early_stopping=True,
    temperature=0.7
)

print("Output:\n" + 50 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("\n{}: {}".format(i, 
                        gpt2tokenizer.decode(beam_output, 
                                             skip_special_tokens=True)))

Output:
--------------------------------------------------

0: Robotics is the domain of the U.S. Department of Homeland Security. The agency is responsible for the security of the United States and its allies, including the United Kingdom, Canada, Australia, New Zealand, and the European Union.



1: Robotics is the domain of the U.S. Department of Homeland Security. The agency is responsible for the security of the United States and its allies, including the United Kingdom, France, Germany, Italy, Japan, and the European Union.


2: Robotics is the domain of the U.S. Department of Homeland Security. The agency is responsible for the security of the United States and its allies, including the United Kingdom, Canada, Australia, New Zealand, the European Union, and the United


In [69]:
# Top-K sampling
tf.random.set_seed(42)  # for reproducible results
beam_output = gpt2.generate(
    input_ids, 
    max_length=50, 
    do_sample=True, 
    top_k=25,
    temperature=2
)

print("Output:\n" + 50 * '-')
print(gpt2tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
--------------------------------------------------
Robotics is the domain of people with multiple careers working with robotics systems. The purpose of Robotics & Machine Learning in Science and engineering research is not necessarily different for any given research type because the results would be much more diverse.


Our team uses


In [71]:
input_ids = gpt2tokenizer.encode('In the dark of the night, there was a ', return_tensors='tf')
# Top-K sampling
tf.random.set_seed(42)  # for reproducible results
beam_output = gpt2.generate(
    input_ids, 
    max_length=200, 
    do_sample=True, 
    top_k=50
)

print("Output:\n" + 50 * '-')
print(gpt2tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
--------------------------------------------------
In the dark of the night, there was a sudden appearance of light.

Sighing, Xiao Chen slowly stood up and looked at Tian Cheng standing over. He took a step to look closely at Tian Cheng's left wrist and frowned.

Lin Feng was startled, and quickly took out a long sword!

Lin Feng didn't understand what sort of sword that Long Fei had wielded in the Black and Crystal Palace!

The Black and Crystal Palace was completely different than his original Black Stone City. Long Fei carried a sword as a souvenir, which had been placed on the back of his father's arm by Tian Cheng.

He drew the sword from his dad's arm again!

The black blade was one of the most valuable weapons within the Black and Crystal Palace. The sword was just as sharp as the sharpest of all weapons, which had been placed on Long Fei's father's arm by the Black Stone City's Black Ice, for him to


In [73]:
# Another sample with a larger model
#gpt2tok_l = GPT2Tokenizer.from_pretrained("gpt2-large")

# add the EOS token as PAD token to avoid warnings
#gpt2_l = TFGPT2LMHeadModel.from_pretrained("gpt2-large", 
#                                         pad_token_id=gpt2tokenizer.eos_token_id)
input_ids = gpt2tok_l.encode('In the dark of the night, there was a ', return_tensors='tf')
# Top-K sampling
tf.random.set_seed(42)  # for reproducible results
beam_output = gpt2_l.generate(
    input_ids, 
    max_length=200, 
    do_sample=True, 
    top_k=25
)

print("Output:\n" + 50 * '-')
print(gpt2tok_l.decode(beam_output[0], skip_special_tokens=True))

Output:
--------------------------------------------------
In the dark of the night, there was a sudden light and a strange sound. I quickly realized that it was some kind of magic or a dream."

"So that's what it is then," I said. She shrugged and said, "I'm not going to say more about it." I smiled at her and said, "Okay!" but she looked up and said, "It's just not for me."

I couldn't keep my smile from showing, so I let out a sigh. She looked very happy about it, which I appreciated, so I said, "Yeah, but I don't think you should get upset."

"I know," she said. Then she looked at me and said, "That's right, I can take it or leave it."

I didn't know what to say to that, so my face fell more and more. "I…" I said. I felt a bit embarrassed with myself. Her eyes narrowed at
